<a href="https://colab.research.google.com/github/the-bucketless/nhl_notebooks/blob/main/nzone_faceoff_powerplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Alison Lukan](https://twitter.com/AlisonL) wrote a piece about [powerplays carrying over into a new period](https://www.nhl.com/kraken/news/analytics-with-alison-seattle-kraken-carryover-power-play/c-327547706).  A common thought is that the clean ice surface would aid the powerplay, but people have noted that it rarely seems to actually benefit the team up a player (Ray Ferraro mentions it every time it comes up in a game he's working).  Alison's work shows the anecdotal evidence seems to hold up with powerplays spanning multiple periods underperforming.  

One thing I didn't see mentioned in Alison's article is a comparison of powerplays after a neutral zone faceoff vs those starting in a new period.  Any powerplay at the start of the period has to start at center ice, while those happening in period get to start in the opponent's end of the rink.  This is one noteworthy advantage to starting in period as opposed to on the fresh sheet, so it's plausible that the advantage being shown could have to do with where the powerplay gets its zone start.  So I figured we'd take a quick look to see if it made a difference.  

We'll look at strictly 5-on-4 situations that have a neutral zone faceoff at some point.  We're going to estimate the end of the powerplay by the last event that takes place in the strength state.  This means we'll be overestimating how effective the powerplays are, but it shouldn't favor either of the two situations we're interested in.  

I feel like someone else has done something like this before, but a quick search on [MetaHockey](https://metahockey.com/) didn't turn up anything.  If anyone knows of previous work I should link to, let me know.  Also, I threw this together pretty quick, so if I've missed something or done something foolish, some light mockery will be tolerated.  

The play-by-play data comes courtesy [Harry Shomer](https://twitter.com/offsides_review).

In [1]:
import pandas as pd

This next code block will print out which season it's currently working on.  It should take a minute to get through all of them.

In [2]:
seasons = []

for year in range(2007, 2021):
    season = f"{year}{year + 1}"

    print(season)

    pbp_url = f"https://hockey-data.harryshomer.com/pbp/nhl_pbp{season}.csv.gz"
    pbp = pd.read_csv(pbp_url, compression="gzip")

    pbp["game_seconds"] = pbp.Seconds_Elapsed + (pbp.Period - 1) * 1200
    pbp["pp_goal"] = (pbp.Event == "GOAL") & (pbp.Strength == "5x4")

    # This is to keep track of where changes in strength state take place.
    # There's some data cleaning that ought to be done to make sure this works properly, but we're ignoring that.
    pbp["strength_id"] = ((pbp.Home_Players != pbp.Home_Players.shift(1)) 
                          | (pbp.Away_Players != pbp.Away_Players.shift(1))).cumsum()
    
    # We need to keep track of the most recent neutral zone faceoff within a given powerplay.
    # To ensure we don't end up with faceoffs from previous powerplays, we're starting this new column with
    # a value of -1 wherever a strength change takes place and NA values elsewhere.
    pbp.loc[pbp.strength_id != pbp.strength_id.shift(1), "faceoff_time"] = -1

    # Next, we add in the times when neutral zone faceoffs on a powerplay occurred.
    is_pp_nz_faceoff = (pbp.Event == "FAC") & (pbp.Ev_Zone == "Neu") & (pbp.Strength.isin(["5x4", "4x5"]))
    pbp.loc[is_pp_nz_faceoff, "faceoff_time"] = pbp.game_seconds

    # This way, we can use a forward fill to get the most recent faceoff for every event.
    # Events without a preceding neutral zone faceoff in their strength state will be denoted by a -1.
    pbp.faceoff_time.ffill(inplace=True)

    powerplay = (
        pbp.loc[pbp.Strength.isin(["5x4", "4x5"])]
        .groupby("strength_id")
        .agg({
            "pp_goal": sum,
            "game_seconds": "last",
            "faceoff_time": "last",
        })
    )

    nzone_pp = powerplay.copy().loc[powerplay.faceoff_time != -1]
    nzone_pp["pp_time"] = nzone_pp.game_seconds - nzone_pp.faceoff_time

    # In case something's gone wrong, we'll get rid of anything with a negative time.
    # If there are any, we should inspect them instead to see what went wrong, but we'll keep on racing through.
    nzone_pp = nzone_pp[nzone_pp.pp_time >= 0]

    is_period_start = nzone_pp.faceoff_time % 1200 == 0
    period_start = nzone_pp.loc[is_period_start]
    in_period = nzone_pp.loc[~is_period_start]

    nzone_pp["is_period_start"] = nzone_pp.faceoff_time % 1200 == 0

    # To make the values a little more interpretable, we're listing them as goals per 2 minutes of powerplay time.
    season_summary = nzone_pp.groupby("is_period_start", as_index=False)[["pp_goal", "pp_time"]].sum()
    season_summary["goals_per_minor"] = season_summary.pp_goal / season_summary.pp_time * 120
    season_summary["season"] = season

    seasons.append(season_summary)

20072008
20082009
20092010
20102011
20112012
20122013
20132014
20142015
20152016
20162017
20172018
20182019
20192020
20202021


After all the work is done, we can inspect how things look in each season we have data for.  For those not reading the code comments, goals_per_minor isn't goals per minor penalty but goals per two minutes of powerplay time.

In [3]:
all_seasons = pd.concat(seasons)
all_seasons

,is_period_start,pp_goal,pp_time,goals_per_minor,season
0,False,406,253052.0,0.192530,20072008
1,True,27,25980.0,0.124711,20072008
0,False,243,94822.0,0.307524,20082009
1,True,31,25896.0,0.143652,20082009
0,False,205,86489.0,0.284429,20092010
1,True,20,23515.0,0.102063,20092010
0,False,226,87292.0,0.310681,20102011
1,True,22,21585.0,0.122307,20102011
0,False,204,81780.0,0.299340,20112012
1,True,22,21578.0,0.122347,20112012


Finally, we can look at the grand totals for all the seasons combined.

In [4]:
totals = all_seasons.groupby("is_period_start")[["pp_goal", "pp_time"]].sum()
totals["goals_per_minor"] = totals.pp_goal / totals.pp_time * 120

totals

,pp_goal,pp_time,goals_per_minor
is_period_start,,,
False,2922,1148396.0,0.305330
True,338,300673.0,0.134897


Based on this, it looks like powerplays at the start of a period still show poorly.  Don't think we can chalk up this one to the zone start.